In [1]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np


2023-11-27 17:10:21.004456: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 17:10:21.052513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# 데이터 로드
data = pd.read_csv("./data/season.csv")
data=data.drop("Unnamed: 0",axis=1)

In [5]:
data["태그"].value_counts()

태그
winter    3624
fall      3600
summer    3567
spring    3483
Name: count, dtype: int64

In [4]:
# 새로운 데이터프레임 생성
data1 = pd.DataFrame()
data2 = pd.DataFrame()
data3 = pd.DataFrame()

# 원래 가사를 세 부분으로 나누어 세 데이터프레임에 추가
for i in range(len(data["가사"])):
    list1 = list(str(data["가사"][i]).split(" "))
    length = len(list1)
    one_third = length // 3
    two_third = 2 * length // 3
    
    first_part = " ".join(list1[:one_third])
    second_part = " ".join(list1[one_third:two_third])
    third_part = " ".join(list1[two_third:])
    
    # 첫 번째 데이터프레임에 추가
    data1.loc[i, "가사"] = first_part
    data1.loc[i, "태그"] = data["태그"][i]
    
    # 두 번째 데이터프레임에 추가
    data2.loc[i, "가사"] = second_part
    data2.loc[i, "태그"] = data["태그"][i]
    
    # 세 번째 데이터프레임에 추가
    data3.loc[i, "가사"] = third_part
    data3.loc[i, "태그"] = data["태그"][i]

# 세 데이터프레임 합치기
data = pd.concat([data1, data2, data3], axis=0)


/tmp/ipykernel_680604/1052554418.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Said you play a little hard to get And I bet it worked before but baby Honestly I dont like this back and forth I know you show me things to make me think its real But you never really tell me how you feel oh Hard to get With me with me What do you drink What do you smoke What do you do on Sundays alone Is it all on you or do you have someone to hold To tell you the truth You aint told me nothing at all Said you keeping your distance from me Then why do you text me cuz I aint ya bestie You know that its more than nothin Feel like ya test me but that doesnt make me down I see what youre doing But hear me when' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data1.loc[i, "가사"] = first_part
/tmp/ipykernel_680604/1052554418.py:19: FutureWarning: Setting an item of incompatible dtype is deprec

In [6]:
data.dropna(subset=['태그'], inplace=True)

In [4]:
# # 가사길이를 앞부분만사용하기위해 100개의 단어까지만 받도록함
# list1=[]
# for i in range (len(data["가사"])):
#     list1=list(data["가사"][i].split(" "))
#     list1= list1[:100]
#     data["가사"][i]=" ".join(list1)

In [9]:
# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')


2023-11-27 17:11:25.109023: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 350.24MiB (rounded to 367248384)requested by op StatelessTruncatedNormalV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-27 17:11:25.109067: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-27 17:11:25.109076: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 7, Chunks in use: 7. 1.8KiB allocated for chunks. 1.8KiB in use in bin. 64B client-requested in use in bin.
2023-11-27 17:11:25.109081: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-11-27 17:11:25.109086: I tensorflow/tsl/framework/bfc

ResourceExhaustedError: Exception encountered when calling layer 'bert' (type TFBertMainLayer).

{{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[119547,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2]

Call arguments received by layer 'bert' (type TFBertMainLayer):
  • input_ids=tf.Tensor(shape=(1, 2), dtype=int32)
  • attention_mask=tf.Tensor(shape=(1, 2), dtype=int32)
  • token_type_ids=tf.Tensor(shape=(1, 2), dtype=int32)
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [10]:
# 감정 태그를 숫자로 매핑
tag_mapping = {tag: idx for idx, tag in enumerate(data['태그'].unique())}
inverse_tag_mapping = {v: k for k, v in tag_mapping.items()}
data['태그_encoded'] = data['태그'].map(tag_mapping)

# 데이터 분할
train,test= train_test_split(data[["가사",'태그_encoded']], test_size=0.2, random_state=42)

In [7]:
train

,가사,태그_encoded
2484,We're a long way from home Haven't seen you in...,2
3016,Does she move like air? Yeah I bet they stop a...,2
532,알기에 우리의 밤이 끝나지 않기만 바래요 눈 속에 핀 꽃처럼 우린 봄을 함께 하진 ...,0
3172,say Can't we just leave and walk away? Walk aw...,2
4734,I miss the,3
...,...,...
974,날 비가 내리는 날에는 나를 생각해줘요 함께 걸었던 거리를 기억해줘요 비가 내리는 ...,0
431,"의미 Date, Just 떳떳하게 너의 앞에 선 나를 상상해온 길었던 시간을 달려서...",0
630,like I’m paralyzed) cause I’ve been staying up...,0
860,How can i forget you girl damage from your smi...,0


In [11]:
data["태그"].unique()

array(['spring', 'summer', 'fall', 'winter'], dtype=object)

In [12]:
x_train = tokenizer(
    text=train.가사.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=test.가사.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [14]:
x_test["input_ids"][200]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([   101,   9257,  77884,   9087,  48387,  18108,   9290,   9937,
       118632,  10622,    100,   8924, 118729,   9004,  11287,   9682,
         9524,  26737, 119221, 103014,   9670, 118775,  10622,   9524,
        14153,   9097,  10622,    100,   9995,  11102,   9309,  86488,
         8996,  14153,   9695,   8982,  11513,   9670,  25387,   9290,
         9730,  12692,  14153,   9248,  93200, 119221,   8984,   9590,
        31401,  31401,   9294,  70162,  68833,   9251,  16985,   8982,
        10459,   9253,  10892,   9590,  31401,  31401,   8996,  14153,
         9638,  30873,  12508,   9246,  17073,   9004,  40364,   9952,
        35866,  11018,   9414,  14871,  22440,  20626,  23466,   9490,
        10892,   8867,   9519,  32537, 118627,  12092,   9955,   9460,
         9555,  10622,   8867,   8984,   9318,  30005,  11287,   9096,
        41605,  85836,   8863,    102], dtype=int32)>

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [26]:
max_len = 100
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert_model(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = tf.keras.layers.Dropout(0.3)(out)
out = Dense(1024, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(4,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
# for training bert our lr must be so small

In [27]:
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy(),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                         

In [29]:
# 가사 앞뒤로 나눠서 2배로 늘린 모델
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습에 EarlyStopping 콜백 적용
train_history = model.fit(
    x={'input_ids': x_train['input_ids'], 'attention_mask': x_train['attention_mask']},
    y=to_categorical(train.태그_encoded),
    validation_data=(
        {'input_ids': x_test['input_ids'], 'attention_mask': x_test['attention_mask']},
        to_categorical(test.태그_encoded)),
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping]
)


Epoch 1/50


/home/ubuntu/miniforge3/envs/tf212_py310/lib/python3.10/site-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


179/179 [==============================] - 34s 113ms/step - loss: 0.9941 - categorical_accuracy: 0.5843 - val_loss: 1.2104 - val_categorical_accuracy: 0.4806
Epoch 2/50
179/179 [==============================] - 18s 98ms/step - loss: 0.8436 - categorical_accuracy: 0.6637 - val_loss: 1.2927 - val_categorical_accuracy: 0.4757
Epoch 3/50
179/179 [==============================] - 19s 105ms/step - loss: 0.7408 - categorical_accuracy: 0.7096 - val_loss: 1.3012 - val_categorical_accuracy: 0.4883
Epoch 4/50
179/179 [==============================] - 17s 98ms/step - loss: 0.6544 - categorical_accuracy: 0.7505 - val_loss: 1.4754 - val_categorical_accuracy: 0.4963
Epoch 5/50
179/179 [==============================] - 17s 98ms/step - loss: 0.5828 - categorical_accuracy: 0.7809 - val_loss: 1.4188 - val_categorical_accuracy: 0.5096
Epoch 6/50
179/179 [==============================] - 19s 105ms/step - loss: 0.5279 - categorical_accuracy: 0.8021 - val_loss: 1.5708 - val_categorical_accuracy: 0.5142


In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(train.태그_encoded),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(test.태그_encoded)),
    epochs=50,
    batch_size=64)

In [17]:
# 예측 결과를 원래 태그로 변환
predictions = model.predict(
   {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_tags = np.argmax(predictions, axis=1)
predicted_tags = [inverse_tag_mapping[tag] for tag in predicted_tags]

print("Predicted Tags:", predicted_tags)


90/90 [==============================] - 4s 22ms/step
Predicted Tags: ['spring', 'fall', 'spring', 'summer', 'spring', 'summer', 'fall', 'summer', 'summer', 'fall', 'winter', 'summer', 'fall', 'summer', 'spring', 'spring', 'fall', 'fall', 'summer', 'winter', 'spring', 'fall', 'spring', 'fall', 'spring', 'summer', 'summer', 'fall', 'spring', 'summer', 'fall', 'spring', 'spring', 'spring', 'winter', 'summer', 'summer', 'fall', 'spring', 'spring', 'winter', 'fall', 'spring', 'spring', 'summer', 'winter', 'fall', 'fall', 'winter', 'spring', 'fall', 'spring', 'fall', 'fall', 'fall', 'fall', 'fall', 'summer', 'summer', 'summer', 'spring', 'summer', 'summer', 'fall', 'fall', 'summer', 'winter', 'summer', 'spring', 'spring', 'winter', 'summer', 'fall', 'spring', 'winter', 'winter', 'summer', 'fall', 'summer', 'spring', 'spring', 'spring', 'spring', 'summer', 'winter', 'summer', 'fall', 'winter', 'winter', 'summer', 'summer', 'summer', 'summer', 'fall', 'spring', 'fall', 'spring', 'fall', 'spri

In [18]:
from sklearn.metrics import classification_report

# 예측 결과와 실제 레이블을 가지고 classification report 생성
y_true = test.태그_encoded
y_pred =np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.43      0.45      0.44       696
           1       0.68      0.49      0.57       707
           2       0.38      0.67      0.49       680
           3       0.48      0.27      0.34       772

    accuracy                           0.46      2855
   macro avg       0.49      0.47      0.46      2855
weighted avg       0.49      0.46      0.46      2855



In [23]:
a=input()
b=tokenizer(
    text=a,
    add_special_tokens=True,
    max_length=200,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

 겨울 아침 


In [24]:
predictions = model.predict({'input_ids': b['input_ids'], 'attention_mask': b['attention_mask']})
predicted_tags = np.argmax(predictions, axis=1)
predicted_tags = [inverse_tag_mapping[tag] for tag in predicted_tags]
print("Predicted Tags:", predicted_tags)

1/1 [==============================] - 0s 37ms/step
Predicted Tags: ['fall']


In [16]:
model.save_weights('./model/time/season2_weights')
model.save('./model/time/season2.h5')